In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import norm, skew 
from scipy import stats #qqplot
import statsmodels.api as sm 
from matplotlib import rcParams
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import tensorflow as tf
print(tf.__version__)


**Loading the datafarme**

In [ ]:
df  = pd.read_csv('../input/productdemandforecasting/Historical Product Demand.csv', parse_dates=['Date'])

In [ ]:
df.head()

**Finding the NAN values**
1. Find the percentage of total NAN value

In [ ]:
print (df.isna().sum())
print((100 * df['Date'].isna().sum()) / len(df))

Since only 1% of data is NAN therefore we will drop all NAN values

In [ ]:
df = df.dropna(axis=0)

 **Visualizign whole data using heatmap**
1. make sure there is no NAN value 

In [ ]:
df.isnull()
sns.heatmap(df.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

**Removing some unnecessary () Found in Order_Demand Column**

In [ ]:
df.sort_values('Date')[10:20] 

df['Order_Demand'] = df['Order_Demand'].str.replace('(',"")
df['Order_Demand'] = df['Order_Demand'].str.replace(')',"")

#Chaning the dataType into int64
df['Order_Demand'] = df['Order_Demand'].astype('int64')

**Visualizing the max order shipped by each warehouse**

In [ ]:
x = df.groupby('Warehouse').sum().sort_values('Order_Demand', ascending = False)
equiv = {3363200396:"Whse_J", 1038024700:"Whse_S", 585071404:"Whse_C",147877431:"Whse_A"}
x["Order"] = x["Order_Demand"].map(equiv)
x.plot.bar(x='Order', y='Order_Demand', rot=0)

**Visualizing all distinct product along with their quantity**

In [ ]:
print ("Total Distinct Proudct Category",len(df['Product_Category'].value_counts()))
rcParams['figure.figsize'] = 50,10
sb.countplot(df['Product_Category'].sort_values(ascending = True))

**Visualizing All Orders By Warehouses**


In [ ]:
rcParams['figure.figsize'] = 16,4
f, axes = plt.subplots(1, 2)
fig3 = sb.boxplot( df['Warehouse'],df['Order_Demand'], ax = axes[0])
fig4 = sb.boxplot( df['Warehouse'], np.log1p(df['Order_Demand']),ax = axes[1])


**Visualizing All Orders by Product Category**


In [ ]:
rcParams['figure.figsize'] = 50,12
df_temp = df.sample(n=20000).reset_index()
fig5 = sb.boxplot( df_temp['Product_Category'].sort_values(),np.log1p(df_temp['Order_Demand']))

**Preparing the Dataset**

In [ ]:

df = pd.read_csv('../input/productdemandforecasting/Historical Product Demand.csv')


df.dropna(axis=0, inplace=True) #remove all rows with na's.
df.reset_index(drop=True)

#Target Feature - Order_Demand
#Removing () from the target feature.
df['Order_Demand'] = df['Order_Demand'].str.replace('(',"")
df['Order_Demand'] = df['Order_Demand'].str.replace(')',"")

#Next step is to change the data type.
df['Order_Demand'] = df['Order_Demand'].astype('int64')

df.drop('Product_Code', inplace=True, axis=1)
df.drop('Warehouse', inplace=True, axis=1)
df.drop('Product_Category', inplace=True, axis=1)

df = df.groupby('Date')['Order_Demand'].sum().reset_index()


#Index the date
df = df.set_index('Date')
df.index #Lets check the index

df.to_csv('./final.csv')

df.head()

**Making Prediction**

In [ ]:
df=pd.read_csv('./final.csv')

df1=df.reset_index()['Order_Demand']

df1

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

print(df1)

In [ ]:
##splitting dataset into train and test split
training_size=int(len(df1)*0.8)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

training_size,test_size

In [ ]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

print(X_train.shape), print(y_train.shape)

print(X_test.shape), print(ytest.shape)

# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=50,batch_size=64,verbose=1)

In [ ]:
### prediction and performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

### Test Data RMSE
math.sqrt(mean_squared_error(ytest,test_predict))

In [ ]:
### Plotting 
# shift train predictions for plotting
look_back=100
trainPredictPlot = numpy.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(df1)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()